In [2]:
import numpy as np
import pandas as pd
from dataclasses import dataclass, field
from typing import Any
from queue import PriorityQueue
import numpy.ma as ma
from collections import namedtuple

In [77]:
monster_t = np.dtype(
    [
        ("id", int),
        ("name", np.bytes_, 20),
        ("symbol", np.byte),
        ("exp_level", int),
        ("exp_points", int),
        ("damage", int),
        ("armor_class", int),
        ("pos", int, (2,)),
        ("hp", int),
        ("max_hp", int),
        ("is_asleep", bool),
    ]
)


In [78]:
MonsterType = namedtuple("monster_type", ["name", "symbol", "exp_level", "exp_points", "armor_class", "damage"])
monster_types = (
    MonsterType(name="kestrel", symbol=ord("K"), exp_level=1, exp_points=2, armor_class=7, damage=4),
    MonsterType(name="emu", symbol=ord("E"), exp_level=1, exp_points=2, armor_class=7, damage=2),
    MonsterType(name="snake", symbol=ord("S"), exp_level=1, exp_points=2, armor_class=5, damage=3),
    MonsterType(name="bat", symbol=ord("B"), exp_level=1, exp_points=2, armor_class=3, damage=2),
    MonsterType(name="hobgoblin", symbol=ord("H"), exp_level=1, exp_points=3, armor_class=5, damage=8),
    MonsterType(name="orc", symbol=ord("O"), exp_level=1, exp_points=5, armor_class=6, damage=8),
    MonsterType(name="zombie", symbol=ord("Z"), exp_level=2, exp_points=8, armor_class=8, damage=8),
)


In [218]:
# "name, symbol, exp_level, exp_points, armor_class, damage"
# fmt: off
_monster_types = (
    ("kestrel  ", ord("K"), 1, 2, 7, 4),
    ("emu",       ord("E"), 1, 2, 7, 2),
    ("snake",     ord("S"), 1, 2, 5, 3),
    ("bat",       ord("B"), 1, 2, 3, 2),
    ("hobgoblin", ord("H"), 1, 3, 5, 8),
    ("orc",       ord("O"), 1, 5, 6, 8),
    ("zombie",    ord("Z"), 2, 8, 8, 8),
)
# fmt: on

rng = np.random.default_rng()  # move to globals
monster_type = rng.choice(_monster_types)


In [219]:
monster_type

array(['zombie', '90', '2', '8', '8', '8'], dtype='<U21')

In [229]:
_monster_type_dtype = np.dtype(
    [
        ("name", "S20"),
        ("symbol", int),
        ("exp_level", int),
        ("exp_points", int),
        ("damage", int),
        ("armor_class", int),
    ]
)

points = np.array(
    [
        ("kestrel", ord("K"), 1, 2, 7, 4),
        ("emu", ord("E"), 1, 2, 7, 2),
        ("snake", ord("S"), 1, 2, 5, 3),
        ("bat", ord("B"), 1, 2, 3, 2),
        ("hobgoblin", ord("H"), 1, 3, 5, 8),
        ("orc", ord("O"), 1, 5, 6, 8),
        ("zombie", ord("Z"), 2, 8, 8, 8),
    ],
    dtype=_monster_type_dtype,
)

In [275]:
_monster_type_dtype = np.dtype(
    [
        ("name", "S20"),
        ("symbol", int),
        ("exp_level", int),
        ("exp_points", int),
        ("damage", int),
        ("armor_class", int),
    ]
)

_monster_types = np.array(
    [
        ("kestrel", ord("K"), 1, 2, 7, 4),
        ("emu", ord("E"), 1, 2, 7, 2),
        ("snake", ord("S"), 1, 2, 5, 3),
        ("bat", ord("B"), 1, 2, 3, 2),
        ("hobgoblin", ord("H"), 1, 3, 5, 8),
        ("orc", ord("O"), 1, 5, 6, 8),
        ("zombie", ord("Z"), 2, 8, 8, 8),
    ],
    dtype=_monster_type_dtype,
)

rng = np.random.default_rng()  # move to globals
monster_type = rng.choice(_monster_types)
exp_level = monster_type["exp_level"]
monster_type, exp_level

(np.void((b'bat', 66, 1, 2, 3, 2), dtype=[('name', 'S20'), ('symbol', '<i8'), ('exp_level', '<i8'), ('exp_points', '<i8'), ('damage', '<i8'), ('armor_class', '<i8')]),
 np.int64(1))

In [283]:
_MAX_MONTERS = 100

_monster_dtype = np.dtype(
    [
        ("name", "S20"),
        ("symbol", int),
        ("exp_level", int),
        ("exp_points", int),
        ("damage", int),
        ("armor_class", int),
        ("pos", int, (2,)),
        ("hp", int),
        ("max_hp", int),
        ("is_asleep", bool),
    ]
)

_monster_type_dtype = np.dtype(
    [
        ("name", "S20"),
        ("symbol", int),
        ("exp_level", int),
        ("exp_points", int),
        ("damage", int),
        ("armor_class", int),
    ]
)

# fmt: off
_monster_types = np.array(
    [
    #    name,        symbol,   exp_level, exp_points, armor_class, damage
        ("kestrel",   ord("K"), 1,         2,          7,           4),
        ("emu",       ord("E"), 1,         2,          7,           2),
        ("snake",     ord("S"), 1,         2,          5,           3),
        ("bat",       ord("B"), 1,         2,          3,           2),
        ("hobgoblin", ord("H"), 1,         3,          5,           8),
        ("orc",       ord("O"), 1,         5,          6,           8),
        ("zombie",    ord("Z"), 2,         8,          8,           8),
    ],
    dtype=_monster_type_dtype
)
# fmt: on

monsters = ma.masked_all(_MAX_MONTERS, dtype=_monster_dtype)


def _get_first_masked_row_index():
    i = monsters.mask["name"].argmax()
    return i if monsters.mask["name"][i] else _MAX_MONTERS


def reset_monsters(n: int):
    monsters[:] = ma.masked
    for i in range(n):
        monster_type = rng.choice(_monster_types)
        exp_level = monster_type["exp_level"]
        hp = rng.integers(1, 9, size=exp_level).sum()
        max_hp = hp
        monster = (*monster_type, (0, 0), hp, max_hp, True)
        monsters[i] = monster


In [49]:
from enum import IntEnum
import numpy as np
import numpy.ma as ma

item_dtype = np.dtype(
    [
        ("item_type", int),
        ("symbol", int),
        ("gold", int),
        ("pos", int)
    ]
)

I = IntEnum("Item", ["ARMOR", "WEAPON", "SCROLL", "POTION", "GOLD", "FOOD"])

class ICOL(IntEnum):
    TYPE = 0
    SYMBOL = 1
    GOLD = 2
    POS = 3

print(ICOL.TYPE, ICOL.SYMBOL, ICOL.GOLD, ICOL.POS)

0 1 2 3


In [51]:
items = ma.masked_all((100, 4), dtype=int)

rng = np.random.default_rng()

items[:] = ma.masked
n = 10
items[:n, ICOL.TYPE] = I.GOLD
items[:n, ICOL.SYMBOL] = ord("*")
items[:n, ICOL.GOLD] = rng.integers(10, 100, size=n)
items[:n, ICOL.POS] = (rng.integers(23, size=n) << 8) + rng.integers(
    80, size=n
)
items

masked_array(
  data=[[5, 42, 20, 299],
        [5, 42, 61, 3902],
        [5, 42, 40, 2117],
        [5, 42, 25, 4930],
        [5, 42, 52, 2838],
        [5, 42, 65, 4671],
        [5, 42, 69, 1071],
        [5, 42, 46, 4630],
        [5, 42, 12, 3360],
        [5, 42, 57, 1351],
        [--, --, --, --],
        [--, --, --, --],
        [--, --, --, --],
        [--, --, --, --],
        [--, --, --, --],
        [--, --, --, --],
        [--, --, --, --],
        [--, --, --, --],
        [--, --, --, --],
        [--, --, --, --],
        [--, --, --, --],
        [--, --, --, --],
        [--, --, --, --],
        [--, --, --, --],
        [--, --, --, --],
        [--, --, --, --],
        [--, --, --, --],
        [--, --, --, --],
        [--, --, --, --],
        [--, --, --, --],
        [--, --, --, --],
        [--, --, --, --],
        [--, --, --, --],
        [--, --, --, --],
        [--, --, --, --],
        [--, --, --, --],
        [--, --, --, --],
        [--, --

In [52]:
len(ICOL)

4

In [356]:
entities = ma.array(rng.integers(10, size=(4, 4)))
entities[0, :] = ma.masked
entities[3, :] = ma.masked
entities[:, 0] = ma.masked
entities[:, 3] = ma.masked
# entities = (entities > 5)*entities
mapped = np.full_like(entities, fill_value=11)
mapped[0,0] = 0
mapped[3,3] = 0
# visible = np.zeros_like(entities)
# visible[1:3,1:3] = 1
# np.where(entities*visible, entities, mapped)

In [53]:
1 + True

2

In [14]:
df = pd.DataFrame({"a": [(1,1),(2,2),(3,3)], "b": [(5,6), (6,7), (7,8)]})
df.at[1, "b"] = pd.NA
df.info()


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 3 entries, 0 to 2
Data columns (total 2 columns):
 #   Column  Non-Null Count  Dtype 
---  ------  --------------  ----- 
 0   a       3 non-null      object
 1   b       2 non-null      object
dtypes: object(2)
memory usage: 180.0+ bytes


In [54]:
from queue import SimpleQueue

    

In [58]:
messages = SimpleQueue()
messages.put(3)
messages.put(2)
while messages.qsize() > 0:
    x = messages.get()
    print(x)
messages.empty()

3
2


True

In [59]:
class ICOL(IntEnum):
    TYPE = 0
    SYMBOL = 1
    GOLD = 2
    POS = 3


In [66]:
from enum import StrEnum, auto
class Color(StrEnum):
    RED = auto()
    GREEN = auto()
    BLUE = auto()

In [91]:
potions = np.array([
    "blindness",
    "confusion",
    "extra healing",
    "gain strength" "hallucination",
    "haste self",
    "healing",
    "magic detection",
    "monster detection",
    "paralysis",
    "poison",
    "raise level",
    "restore strength",
    "see invisible",
    "thirst quenching",
])

colors = np.array([
    "amber",
    "aquamarine",
    "black",
    "blue",
    "brown",
    "clear",
    "crimson",
    "cyan",
    "ecru",
    "gold",
    "green",
    "grey",
    "magenta",
    "orange",
    "pink",
    "plaid",
    "purple",
    "red",
    "silver",
    "tan",
    "tangerine",
    "topaz",
    "turquoise",
    "vermilion",
    "violet",
    "white",
    "yellow",
])

In [104]:
A = np.arange(16).reshape(4, 4)
A[A[:,0] > 5, [2, 3]]

array([10, 15])

In [103]:
S = {(y << 8) + x for y in range(23) for x in range(1, 79)}

array([False, False,  True,  True])

In [102]:
A   

array([[ 0,  1,  2,  3],
       [ 4,  5,  6,  7],
       [ 8,  9, 10, 11],
       [12, 13, 14, 15]])